# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
import re
import sqlite3
import numpy as np
import pandas as pd
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import classification_report, accuracy_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

[nltk_data] Downloading package punkt to /Users/lawrence/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lawrence/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/lawrence/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')

# connect to the database
conn = sqlite3.connect('/Users/lawrence/documents/_Data Scientist/Part 9 - Data Engineering/dsnd4-p5-disaster-response-pipelines/data/DisasterResponse.db')

# run a query
df = pd.read_sql('SELECT * FROM disaster', conn)
X = df['message']
Y = df.iloc[:,4:]
category_names = list(Y)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    # tokenize text
    tokens = word_tokenize(text)
    
    # initiate lemmatizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
- You'll find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),
        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# train classifier
pipeline.fit(X_train, Y_train)

# predict on test data
Y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
for i in range(36):
    print(category_names[i], '\n', classification_report(Y_test.iloc[:,i], Y_pred[:,i]))

related 
              precision    recall  f1-score   support

          0       0.67      0.41      0.51      1610
          1       0.83      0.93      0.88      4944

avg / total       0.79      0.81      0.79      6554

request 
              precision    recall  f1-score   support

          0       0.91      0.97      0.94      5412
          1       0.77      0.56      0.65      1142

avg / total       0.89      0.89      0.89      6554

offer 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00      6527
          1       0.00      0.00      0.00        27

avg / total       0.99      0.99      0.99      6554

aid_related 
              precision    recall  f1-score   support

          0       0.75      0.87      0.81      3860
          1       0.76      0.59      0.66      2694

avg / total       0.75      0.75      0.75      6554

medical_help 
              precision    recall  f1-score   support

          0       0.94      0

### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters = {
#         'text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
#         'text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
#         'text_pipeline__vect__max_features': (None, 5000, 10000),
         'text_pipeline__tfidf__use_idf': (True, False),
         'clf__estimator__n_estimators': [10, 50, 100],
         'clf__estimator__learning_rate': [0.1, 1, 5],
}
    
cv = GridSearchCV(pipeline, param_grid=parameters)

In [8]:
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'text_pipeline__tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [10, 50, 100], 'clf__estimator__learning_rate': [0.1, 1, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
# predict on test data
Y_pred = cv.predict(X_test)

In [10]:
for i in range(36):
    category = category_names[i]
    accuracy = accuracy_score(Y_test.iloc[:,i], Y_pred[:,i])
    precision = precision_score(Y_test.iloc[:,i], Y_pred[:,i])
    recall = recall_score(Y_test.iloc[:,i], Y_pred[:,i])
    print(category)
    print("\tAccuracy: %.4f\tPrecision: %.4f\t Recall: %.4f\n" % (accuracy, precision, recall))

related
	Accuracy: 0.8073	Precision: 0.8376	 Recall: 0.9235

request
	Accuracy: 0.8982	Precision: 0.7739	 Recall: 0.5876

offer
	Accuracy: 0.9940	Precision: 0.0000	 Recall: 0.0000

aid_related
	Accuracy: 0.7650	Precision: 0.7574	 Recall: 0.6303

medical_help
	Accuracy: 0.9254	Precision: 0.5530	 Recall: 0.2824

medical_products
	Accuracy: 0.9536	Precision: 0.5676	 Recall: 0.3191

search_and_rescue
	Accuracy: 0.9764	Precision: 0.4638	 Recall: 0.2133

security
	Accuracy: 0.9808	Precision: 0.2353	 Recall: 0.0342

military
	Accuracy: 0.9692	Precision: 0.5912	 Recall: 0.3568

child_alone
	Accuracy: 1.0000	Precision: 0.0000	 Recall: 0.0000

water
	Accuracy: 0.9599	Precision: 0.7178	 Recall: 0.6209

food
	Accuracy: 0.9439	Precision: 0.7885	 Recall: 0.6681

shelter
	Accuracy: 0.9442	Precision: 0.7494	 Recall: 0.5559

clothing
	Accuracy: 0.9873	Precision: 0.6364	 Recall: 0.4158

money
	Accuracy: 0.9792	Precision: 0.5326	 Recall: 0.3451

missing_people
	Accuracy: 0.9869	Precision: 0.3448	 Recall:

In [11]:
cv.best_params_

{'clf__estimator__learning_rate': 1,
 'clf__estimator__n_estimators': 100,
 'text_pipeline__tfidf__use_idf': True}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [12]:
filename = 'classifier.pkl'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.